# Employee Project Management and Performance Analysis using NumPy and Pandas
_Revised version (NumPy + Pandas enforced) - 2025-08-31 13:41 UTC_

This notebook demonstrates **data analysis and transformation** using both **NumPy** and **Pandas**.

We solve 10 tasks step by step.  
Each task includes:  
- Markdown explanation  
- Python code with **NumPy + Pandas**  
- Clear outputs

---


## Task 1
Load data from CSV into Pandas DataFrames.

In [ ]:
import pandas as pd
import numpy as np

df_project = pd.read_csv(r"/mnt/data/Project.csv")
df_employee = pd.read_csv(r"/mnt/data/Employee.csv")
df_seniority = pd.read_csv(r"/mnt/data/Seniority.csv")

display(df_project.head())
display(df_employee.head())
display(df_seniority.head())

## Task 2
Fill missing Cost values using **NumPy running average**.

In [ ]:
costs = df_project["Cost"].astype(float).values
for i in range(len(costs)):
    if np.isnan(costs[i]):
        costs[i] = np.nanmean(costs[:i])
df_project["Cost"] = costs
display(df_project)

## Task 3
Split Name into First and Last using Pandas string ops.

In [ ]:
names = df_employee["Name"].str.split(" ", n=1, expand=True)
df_employee["First Name"], df_employee["Last Name"] = names[0], names[1]
df_employee = df_employee.drop(columns=["Name"])
display(df_employee)

## Task 4
Join Project, Employee, and Seniority into Final DataFrame.

In [ ]:
final_df = df_project.merge(df_employee, on="ID").merge(df_seniority, on="ID")
display(final_df.head())

## Task 5
Add Bonus column: 5% of cost if Finished.

In [ ]:
bonus = np.where(final_df["Status"].eq("Finished"), 0.05*final_df["Cost"], 0.0)
final_df["Bonus"] = bonus
display(final_df.head())

## Task 6
Demote designation for Failed projects; drop >4.

In [ ]:
final_df.loc[final_df["Status"]=="Failed", "Designation Level"] += 1
final_df = final_df[final_df["Designation Level"] <= 4].reset_index(drop=True)
display(final_df.head())

## Task 7
Prefix Mr./Mrs. using NumPy where; drop Gender.

In [ ]:
first_names = np.where(final_df["Gender"]=="M",
                       "Mr. " + final_df["First Name"],
                       "Mrs. " + final_df["First Name"])
final_df["First Name"] = first_names
final_df = final_df.drop(columns=["Gender"])
display(final_df.head())

## Task 8
Promote (reduce level) for Age > 29 using NumPy boolean mask.

In [ ]:
mask = final_df["Age"].values > 29
final_df.loc[mask, "Designation Level"] -= 1
display(final_df.head())

## Task 9
Aggregate Cost by ID + First Name.

In [ ]:
TotalProjCost = (final_df.groupby(["ID","First Name"], as_index=False)["Cost"]
                           .sum()
                           .rename(columns={"Cost":"TotalProjCost"}))
display(TotalProjCost)

## Task 10
Employees with 'o' in City name.

In [ ]:
mask = final_df["City"].str.contains("o", case=False).values
city_with_o = final_df[mask]
display(city_with_o)